In [1]:
import face_recognition as fr
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd
import sklearn
import math
import pickle #Estudar sobre
from flask import Flask, render_template, Response
print("Import sem erros")


def resizeImage(image, **kwargs):
    if 'scale' in kwargs.keys():
        scale = kwargs.get('scale')
        x = int(image.shape[0] * scale)
        y = int(image.shape[1] * scale)
    return cv2.resize(image, dsize=(x, y), interpolation=cv2.INTER_AREA)

def compare_images(im1, im2):
    plt.Figure(figsize=(10, 10))
    plt.subplot(121)
    plt.imshow(im1)
    plt.subplot(122)
    plt.imshow(im2)

def load_image(path, gray=True):
    im = cv2.imread(path)
    if gray: im = cv2.cv2tColor(im, cv2.COLOR_BGR2RGB)
    return im

def flat_images(images):
    #Flatten the images array
    n, m = images[0].shape
    k = len(images)
    flat_images = np.zeros((n * m, k), dtype='uint8').T
   
    for i in range(k):
        flat_images[i] = images[i].flatten()
    return flat_images


def plotSample(images, labels):
    size = len(images)
    plot_size = np.array(list((3,4))) * 4
    plt.figure(figsize=tuple(plot_size))
    for i in range(12):
        plt.subplot(3,4, i+1)
        plt.imshow(images[i])
        plt.yticks([])
        plt.xticks([])
        plt.xlabel(labels[i])
    plt.show()
   
#Lmebrar que esta recebe a predição com distância = True
#Errado
def get_labels(labels, knn_pred):
   names = []
   index_pred = [i for i in knn_pred[1][0]]
   for i in index_pred:
       names.append(labels[i])
   return names

def read_dir(directory, for_one=False, retrieve_one_image=False):
    path = []
    images = []
    data = []
    
    if not(for_one):
        for sub_folder in os.listdir(directory):
            for filename in os.listdir(os.path.join(directory, sub_folder)):
                image_path = directory+'/'+sub_folder+'/'+filename
                data.append({"Name": sub_folder, 'Path': image_path})
                if retrieve_one_image:
                    break
    else:
        for image in os.listdir(directory):
            image_path = os.path.join(directory,image)
            data.append({"Name": directory.split('/').pop(), "Path":image_path})            
     
    df = pd.DataFrame(data)
    return df

def read_image(path):
    images = []
    for x in path:
        im = cv2.imread(x)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        images.append(im)
    return images


def knn_train(X_train, y_train, model_save_path=None, threads=-1):
    from sklearn.neighbors import KNeighborsClassifier
    n_neighbors = int(math.sqrt(len(X_train))) #Sera susbtituído pelo elbow method
    
    model = KNeighborsClassifier(n_neighbors= n_neighbors, weights='distance', algorithm='auto', p=2, metric='minkowski', n_jobs = threads)
    model.fit(X_train, y_train)
    if model_save_path:
        save_binary(model, model_save_path)
    
    return model

def knn_predict_(image, model_path=None, verbose=False):
    if model_path:
        with open(model_path, 'rb') as f:
            model = pickle.load(f)
   
    y_pred = model.kneighbors(image, n_neighbors=1)
   
    return y_pred

def save_binary(model, path):
    with open(path, 'wb') as f:
        pickle.dump(model, f)

def load_binary(path):
    with open(path, 'rb') as f:
        model = pickle.load(f)
    return model

def unpaking_array(*args):
    new_list = list
    
    
def first_train(train_dir, model_save_path):
    df = read_dir(train_dir, retrieve_one_image=True)
    df = df.iloc[:1500, :] #Here because of the Dataset 
    df['Image'] = read_image(df['Path'])
    print(df.head(10))

    df["Face Location"] = df["Image"].apply(lambda x: fr.face_locations(x, 2, model="hog"))
    df["Face Encoding"] = df.apply(lambda x: fr.face_encodings(x["Image"], x["Face Location"]), axis=1)
    print(df.head(10))

    # model = knn_train([x[0] for x in df['Face Encoding']], df['Name'].values, model_save_path)
    model = [x for x in df['Face Encoding']]
    return model, df

    


Import sem erros


In [2]:
# Realizando leitura do dataset

# if __name__ == '__main__':
model_save_path = "./knn_model.clf"
train_dir = "archive/lfw-deepfunneled"
df_save_path = 'bkp/dataset_example.pkl'


# model, df = first_train(train_dir, model_save_path)


In [13]:

df = read_dir(train_dir, retrieve_one_image=True)
df = df.iloc[:1500, :] #Here because of the Dataset 
df['Image'] = read_image(df['Path'])
print(df.head(10))

                Name                                               Path  \
0        Hugo_Colace  archive/lfw-deepfunneled/Hugo_Colace/Hugo_Cola...   
1      Alexis_Bledel  archive/lfw-deepfunneled/Alexis_Bledel/Alexis_...   
2        Jim_Schwarz  archive/lfw-deepfunneled/Jim_Schwarz/Jim_Schwa...   
3  Patti_Balgojevich  archive/lfw-deepfunneled/Patti_Balgojevich/Pat...   
4  Yekaterina_Guseva  archive/lfw-deepfunneled/Yekaterina_Guseva/Yek...   
5        Bruce_Arena  archive/lfw-deepfunneled/Bruce_Arena/Bruce_Are...   
6      Harland_Braun  archive/lfw-deepfunneled/Harland_Braun/Harland...   
7         Ken_Wharfe  archive/lfw-deepfunneled/Ken_Wharfe/Ken_Wharfe...   
8    Anatoliy_Kinakh  archive/lfw-deepfunneled/Anatoliy_Kinakh/Anato...   
9    Ronald_Young_Jr  archive/lfw-deepfunneled/Ronald_Young_Jr/Ronal...   

                                               Image  
0  [[[143, 137, 87], [143, 137, 87], [144, 138, 8...  
1  [[[77, 46, 51], [76, 45, 50], [74, 45, 49], [7...  
2  [[[0, 

In [21]:
df["Face Location"] = df["Image"].apply(lambda x: fr.face_locations(x, 2, model="hog"))
df["Face Encoding"] = df.apply(lambda x: fr.face_encodings(x["Image"], x["Face Location"]), axis=1)
df.to_pickle(df_save_path) #parquet does not support 2D arrays
print("Fim encode")


Fim encode


In [40]:
df.to_pickle(df_save_path)

In [3]:
df = pd.read_pickle(df_save_path)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           1500 non-null   object
 1   Path           1500 non-null   object
 2   Image          1500 non-null   object
 3   Face Location  1500 non-null   object
 4   Face Encoding  1500 non-null   object
dtypes: object(5)
memory usage: 58.7+ KB


In [6]:
df['Face Encoding'].values

array([list([array([-1.91417083e-01,  1.95971265e-01,  2.98056640e-02, -1.40309753e-03,
              -1.62832886e-01,  5.73441423e-02,  2.68988498e-02, -7.83351064e-02,
               1.19494982e-01,  2.48833857e-02,  2.14858279e-01, -1.61701068e-02,
              -3.00058275e-01, -1.54402955e-02, -8.53237063e-02,  7.82236904e-02,
              -1.30104274e-01, -1.51656300e-01, -1.30642578e-01, -6.25630170e-02,
               1.28162161e-01,  3.90405729e-02, -5.65032102e-02,  6.30049407e-02,
              -2.13056833e-01, -3.15944254e-01, -7.68395737e-02, -1.44707244e-02,
               8.17304775e-02, -1.40941784e-01, -9.08156857e-04,  1.73249207e-02,
              -1.96565345e-01, -4.75858562e-02,  3.14136371e-02,  6.17821515e-02,
              -1.36305420e-02, -5.47061153e-02,  2.33899742e-01, -3.51284072e-03,
              -1.41272202e-01,  8.32951069e-02,  1.25700638e-01,  3.24241579e-01,
               1.36166394e-01, -1.42994476e-02,  2.39175074e-02, -5.41552939e-02,
          

In [35]:
df_2 = pd.DataFrame()
df_2["Face Encoding"] = df.apply(lambda x: fr.face_encodings(x["Image"], x["Face Location"]), axis=1)

FIM


In [58]:
f_len = []
for x in df_2['Face Encoding']:
    df_2['Len Row'] = df_2.apply(lambda x: len(x['Face Encoding']), axis=1)


In [4]:
df.iloc[12]
test_m = zip(df['Face Location'][12], df['Face Encoding'][12])
test_o = zip(df['Face Location'][13], df['Face Encoding'][13])


In [19]:
# for x in range(df.shape):
df.shape

(1500, 5)

In [ ]:
model = knn_train()...
y_pred = model.predict(frame_encoded)
candidate = df[df['Name'] == y_pred]
face_distance = can


